In [ ]:
# 查看BROAD数据集目录
!ls -l /mnt/BROAD-datasets/video/
# 查看文件个数,以training data举例
!ls /mnt/BROAD-datasets/video/training |wc -l
!ls /mnt/BROAD-datasets/video/validation |wc -l
!ls /mnt/BROAD-datasets/video/testing |wc -l
# 查看meta.json
# !cat /mnt/BROAD-datasets/video/meta.json

In [ ]:
# make a new dir, when the server is closed, the data saved here still exist
!mkdir -p /home/kesci/work/Broad/INFO
# copy meta data to the work dir
!cp /mnt/BROAD-datasets/video/meta.json /home/kesci/work/Broad/INFO/
# show the files and dic in the path,
!ls -l /home/kesci/work/Broad/INFO/
# show the memory of each dic
!du -h

In [ ]:
!ls -l /home/kesci/work/
!du -ah

In [ ]:
import cPickle
import sys
import json
import pandas as pd
import numpy as np
import os

def load_json(file):  ## load json file and return data
    with open(file) as json_file:
        data = json.load(json_file)
        return data

def getDatasetDict():
    json_data= load_json("/home/kesci/work/Broad/INFO/meta.json")
    database=json_data['database']  # database
    train_dict={}  # format: {id:{'annotations':[{"segment":[a, b]}, {"segment":[c, d]}]} }
    val_dict={}
    test_dict={}  
    for video_name in database.keys():
        video_info=database[video_name]  # video id
        video_new_info={}
        video_subset=video_info["subset"]
        video_new_info['annotations']=video_info['annotations']
        if video_subset=="training":
            train_dict[video_name]=video_new_info
        elif video_subset=="validation":
            val_dict[video_name]=video_new_info
        elif video_subset=="testing":
            test_dict[video_name]=video_new_info
    return train_dict,val_dict,test_dict

In [ ]:
train_dict, val_dict, test_dict = getDatasetDict()  # all the datasets in meta.json
print(len(train_dict))
print(len(val_dict))
print(len(test_dict))

json_data= load_json("/home/kesci/work/Broad/INFO/meta.json")
database=json_data['database']

train_dir = '/mnt/BROAD-datasets/video/training'
val_dir = '/mnt/BROAD-datasets/video/validation'
test_dir = '/mnt/BROAD-datasets/video/testing'

train_id_list = [name.split('.')[0] for name in os.listdir(train_dir)]  # train video id list in first stage (124)
val_id_list = [name.split('.')[0] for name in os.listdir(val_dir)]  # val video id list in first stage(115)
test_id_list = [name.split('.')[0] for name in os.listdir(test_dir)] # test video id list in first stage(114)
assert len(train_id_list)==124
assert len(val_id_list)==115
assert len(test_id_list)==114

train_gt_dict = {key: value for key, value in train_dict.items() if key in train_id_list}  # train gt dict
val_gt_dict = {key: value for key, value in val_dict.items() if key in val_id_list}  # val gt dict
# test_gt_list = {key: value for key, value in test_dict.items() if key in test_id_list}
assert len(val_gt_dict)==115
assert len(train_gt_dict)==124
# assert len(test_gt_list)==114

In [ ]:
train_dir = '/mnt/BROAD-datasets/video/training'
val_dir = '/mnt/BROAD-datasets/video/validation'
test_dir = '/mnt/BROAD-datasets/video/testing'

train_data_dic = dict()
train_label_dic =dict()
val_data_dic = dict()
val_label_dic = dict()

train_feature_length_list = []  # frame of each video
train_video_id_list = []  # id of each video
train_segment_length_list = []  # length of segment
train_gt_length_list = []  # number of true frame

for video_id in train_id_list:
    train_video_id_list.append(video_id)
    file = os.path.join(train_dir, '{}.pkl'.format(video_id))
    feature = pd.read_pickle(file)
    feature_len = len(feature)
    train_feature_length_list.append(feature_len)
    train_seg = np.array([_['segment'] for _ in train_gt_dict[video_id]['annotations']]).astype(np.int)
    train_segment_length_list.append(len(train_seg))
#     print(train_seg)
    label = np.zeros(feature_len)
    for seg in train_seg:
        label[seg[0]:seg[1]] = 1
    train_gt_length_list.append(sum(label==1))
    
    sample_inds = np.array(range(0, feature_len, 10))
    train_data_dic[video_id] = feature[sample_inds]
    train_label_dic[video_id] = label[sample_inds]
    
# df=pd.DataFrame()
# df['video_id'] = video_id_list
# df['feature_length'] = feature_length_list
# df['segment_length'] = segment_length_list
# df['gt_length'] = gt_length_list
# df.to_csv("/home/kesci/work/Broad/INFO/train_video_info.csv",index=False)
print('done!')

In [ ]:
val_feature_length_list = []  # frame of each video
val_video_id_list = []  # id of each video
val_segment_length_list = []  # length of segment
val_gt_length_list = []  # number of true frame

for video_id in val_id_list:
    val_video_id_list.append(video_id)
    file = os.path.join(val_dir, '{}.pkl'.format(video_id))
    feature = pd.read_pickle(file)
    feature_len = len(feature)
    val_feature_length_list.append(feature_len)
    val_seg = np.array([_['segment'] for _ in val_gt_dict[video_id]['annotations']]).astype(np.int)
    val_segment_length_list.append(len(val_seg))
#     print(train_seg)
    label = np.zeros(feature_len)
    for seg in val_seg:
        label[seg[0]:seg[1]] = 1
    val_gt_length_list.append(sum(label==1))
    
    val_data_dic[video_id] = feature
    val_label_dic[video_id] = label
    
# df=pd.DataFrame()
# df['video_id'] = video_id_list
# df['feature_length'] = feature_length_list
# df['segment_length'] = segment_length_list
# df['gt_length'] = gt_length_list
# df.to_csv("/home/kesci/work/Broad/INFO/val_video_info.csv",index=False)
print('done!')

In [ ]:
train_data = np.concatenate([train_data_dic[_] for _ in train_id_list])
print(train_data.shape)
print(type(train_data))
train_label = np.concatenate([train_label_dic[_] for _ in train_id_list])
print(train_label.shape)
print(type(train_label))

In [ ]:
print(val_id_list==val_video_id_list )

val_infer_data = val_data_dic[val_id_list[0]]
val_infer_label = val_label_dic[val_id_list[0]]

print(val_infer_label)
# print(val_infer_data)
print(val_infer_label[170:220])
infer_data = val_infer_data[170:220, :]
infer_label = val_infer_label[170:220]

In [ ]:
import paddle.v2 as paddle

def multilayer_perception(img):
    hidden1 = paddle.layer.fc(input=img, 
                              size=256, 
                              act=paddle.activation.Relu())
    hidden2 = paddle.layer.fc(input=hidden1, 
                           size=512, 
                           act=paddle.activation.Relu())
    predict = paddle.layer.fc(input=hidden2, 
                              size=2, 
                              act=paddle.activation.Softmax())
    return predict

paddle.init(use_gpu=False, trainer_count=1)
features = paddle.layer.data(name='feature', 
                          type=paddle.data_type.dense_vector(2048))
labels = paddle.layer.data(name='label', 
                          type=paddle.data_type.integer_value(2))
predict = multilayer_perception(features)
cost = paddle.layer.classification_cost(input=predict, label=labels)
parameters = paddle.parameters.create(cost)
optimizer = paddle.optimizer.Momentum(
    learning_rate=0.1/128.0, 
    momentum=0.9, 
    regularization=paddle.optimizer.L2Regularization(rate=0.0005*128))
trainer = paddle.trainer.SGD(cost=cost, 
                             parameters=parameters, 
                             update_equation=optimizer)

def feature_reader_creator(feature, label):
    def reader():
        for i in xrange(len(feature)):
            yield feature[i, :], int(label[i]) # a single entry of data is created each time
    return reader

from paddle.v2.plot import Ploter

train_title = "Train cost"
test_title = "Test cost"
cost_ploter = Ploter(train_title, test_title)

step = 0

# event_handler to plot a figure
def event_handler_plot(event):
    global step
    if isinstance(event, paddle.event.EndIteration):
        if step % 100 == 0:
            cost_ploter.append(train_title, step, event.cost)
            cost_ploter.plot()
        step += 1
#     if isinstance(event, paddle.event.EndPass):
#         # save parameters
#         with open('params_pass_%d.tar' % event.pass_id, 'w') as f:
#             trainer.save_parameter_to_tar(f)

#         result = trainer.test(reader=paddle.batch(
#             paddle.dataset.mnist.test(), batch_size=128))
#         cost_ploter.append(test_title, step, result.cost)

def event_handler(event):
    if isinstance(event, paddle.event.EndIteration):
        if event.batch_id % 100 == 0:
            print "\nPass %d, Batch %d, Cost %f, %s" % (
                event.pass_id, event.batch_id, event.cost, event.metrics)
        else:
            sys.stdout.write('.')
            sys.stdout.flush()
#     if isinstance(event, paddle.event.EndPass):
#         # save parameters
#         with open('params_pass_%d.tar' % event.pass_id, 'w') as f:
#             trainer.save_parameter_to_tar(f)

#         result = trainer.test(
#             reader=paddle.batch(
#                 paddle.dataset.cifar.test10(), batch_size=128),
#             feeding=feeding)
#         print "\nTest with Pass %d, %s" % (event.pass_id, result.metrics)

In [ ]:
feeding={'feature': 0,'label': 1}

reader = feature_reader_creator(train_data, train_label)
# shuffle_reader = paddle.reader.shuffle(reader,buf_size= 32)
trainer.train(paddle.batch(reader, batch_size=32), feeding=feeding, event_handler=event_handler_plot, num_passes=120)

In [ ]:
test_data = []
for i in range(len(val_infer_data)):
    test_data.append(val_infer_data[i,:])
probs = paddle.infer(
    output_layer=predict, parameters=parameters, input=test_data)
lab = np.argsort(-probs) # probs and lab are the results of one batch data   

In [ ]:
pred_label = lab[:, 0]
for i in range(len(val_infer_label) / 35):
    print(pred_label[i*35:i*35+35])
    print(val_infer_label[i*35:i*35+35].astype(int))
    print('*'*10 + str(i))

In [ ]:
train_video_info = pd.read_csv('/home/kesci/work/Broad/INFO/train_video_info.csv')
train_video_info['negetive_length'] = train_video_info['feature_length'] - train_video_info['gt_length']
train_video_info.head(12)

In [ ]:
print(train_video_info.gt_length.sum())
print(train_video_info.feature_length.sum())
train_video_info.gt_length.describe()